# 迁移学习
```markdown
YOLOE 的训练是基于 prompt learning 思想：

模式	   含义	                         特点
-----------------------------------------------------------------------------------------------
Linear   Probing（PE-only）	       只训练最后一层 prompt embedding	快速收敛，避免过拟合，适合小数据
-----------------------------------------------------------------------------------------------
Full     Tuning（PE+Backbone 全部）     更高精度，训练时间更长，需要更多数据
-----------------------------------------------------------------------------------------------
```

##  Linear Probing（只训练 PE）
类似于固定主干，仅训练分类层或 prompt embedding，等于微调最后一层。

`python train_pe.py`

特征：
- 模型权重几乎全部冻结；
- 只训练 PE（Prompt Embedding）模块；
- 速度快，适合小样本场景；
- 通常先使用这个方式 warm-up 微调。

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4" 

# 设置 HTTP 和 HTTPS 代理
os.environ["http_proxy"] = "***"
os.environ["https_proxy"] = "***"

In [4]:
from ultralytics import YOLOE
from ultralytics.models.yolo.yoloe.train_pe import YOLOEPETrainer, YOLOEPESegTrainer
import os
from ultralytics.nn.tasks import guess_model_scale
from ultralytics.utils import yaml_load, LOGGER
import torch

os.environ["PYTHONHASHSEED"] = "0"

配置路径与模型 scale

In [5]:
# data = "ultralytics/cfg/datasets/glass.yaml"
# data = "ultralytics/cfg/datasets/coco128.yaml"
# data = "ultralytics/cfg/datasets/coco.yaml"
data = "ultralytics/cfg/datasets/custom.yaml"

model_path = "yoloe-v8l-seg.yaml"


scale = guess_model_scale(model_path)
cfg_dir = "ultralytics/cfg"
default_cfg_path = f"{cfg_dir}/default.yaml"
extend_cfg_path = f"{cfg_dir}/coco_{scale}_train.yaml"
defaults = yaml_load(default_cfg_path)
extends = yaml_load(extend_cfg_path)
assert(all(k in defaults for k in extends))  # 保证扩展配置键都存在
LOGGER.info(f"Extends: {extends}")

Extends: {'overlap_mask': True, 'scale': 0.9, 'mixup': 0.15, 'copy_paste': 0.3}


In [17]:
# scale, defaults, extends

加载模型并构造 Prompt Embedding

In [8]:
model = YOLOE("./pretrain/yoloe-v8l-seg.pt")
# Ensure pe is set for classes
names = list(yaml_load(data)['names'].values())
tpe = model.get_text_pe(names)
# pe_path = "glass-pe.pt"
# pe_path = "coco128-pe.pt"
# pe_path = "coco-pe.pt"
pe_path = "custom-pe.pt"
torch.save({"names": names, "pe": tpe}, pe_path)

Build text model mobileclip:blt


In [9]:
len(names), tpe.shape

(7, torch.Size([1, 7, 512]))

In [10]:
names

['bracket component',
 'sliding sleeve',
 'elastic bridge',
 'fastening screw',
 'fastening nut',
 'small adjustment block',
 'large adjustment block']

构建冻结列表（freeze 主干网络）

In [11]:
head_index = len(model.model.model) - 1  # 获取最后一层索引
freeze = [str(f) for f in range(0, head_index)]  # 冻结前面所有层

# 进一步冻结检测头部除 prompt 部分
for name, child in model.model.model[-1].named_children():
    if 'cv3' not in name:
        freeze.append(f"{head_index}.{name}")

freeze.extend([f"{head_index}.cv3.0.0", f"{head_index}.cv3.0.1", 
               f"{head_index}.cv3.1.0", f"{head_index}.cv3.1.1", 
               f"{head_index}.cv3.2.0", f"{head_index}.cv3.2.1"])


- data	训练数据 yaml 路径
- epochs	训练轮数（这里只训练 10 轮）
- close_mosaic	关闭 mosaic 的 epoch（用于提升精度）
- batch	批量大小（128 适用于多卡）
- optimizer	使用 AdamW（适合 Prompt Learning）
- trainer	使用 YOLOEPESegTrainer，专用于 Prompt Embedding 训练
- freeze	冻结大部分层，仅训练 prompt
- train_pe_path	提前生成的 PE 参数路径

In [16]:
model.train(data=data, epochs=50, close_mosaic=5, batch=32, 
            optimizer='AdamW', lr0=1e-3, warmup_bias_lr=0.0, \
            weight_decay=0.025, momentum=0.9, workers=4, \
            device="0", **extends, \
            trainer=YOLOEPESegTrainer, freeze=freeze, train_pe_path=pe_path)

### 结果验证

`/root/project/research/Yolo/ultralytics/runs/segment/train6`

In [13]:
from ultralytics import YOLOE

In [15]:
# 加载训练好的模型
import torch
pe_data = torch.load("custom-pe.pt")
names = pe_data["names"]
print(f"names: {names}")
pe = pe_data["pe"]
print(f"pe: {pe.shape}")
model = YOLOE("/root/project/research/Yolo/ultralytics/runs/segment/train7/weights/best.pt")
model.set_classes(names, model.get_text_pe(names))

# model = YOLOE("/root/project/research/Yolo/ultralytics/runs/segment/train6/weights/best.pt")  # 或你自定义保存的路径

# 推理图片
results = model.predict(source='/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp', save=True)

# # 打印结果
# for result in results:
#     print(result.boxes)       # 检测框
#     print(result.masks)       # 分割掩膜（如果是 segment 模型）
#     print(result.probs)       # 类别概率（可选）

names: ['bracket component', 'sliding sleeve', 'elastic bridge', 'fastening screw', 'fastening nut', 'small adjustment block', 'large adjustment block']
pe: torch.Size([1, 7, 512])
Build text model mobileclip:blt

image 1/1 /root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp: 640x640 2 bracket components, 2 sliding sleeves, 2 elastic bridges, 43.5ms
Speed: 21.0ms preprocess, 43.5ms inference, 46.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /root/project/research/Yolo/ultralytics/runs/segment/predict24


## Full Tuning（全部可训练）
对整个网络做 end-to-end 的训练，性能最好但训练时间长

`python train_pe_all.py`

特征：
- 全参数可训练；
- 适用于大数据量的场景；
- 如果模型为 s 级别（YOLOE-v8s），建议将 epochs 改为 160：